<h3>In This notebook we will use XgBoost . 
Few points that we will take care of :
    1. No need to handle null values for XgBoost. In Fact Xgboost can be more benefecial from null entries
  
   
    
  <p>
   <h6> Result Summary :
    
    1. Accuracy 90 on test , when drop NAN above 50 and core more than 70
    

In [1]:
import pandas as pd;
from sklearn.ensemble import RandomForestClassifier;
import matplotlib.pyplot as plt;
import numpy as np;
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [2]:
train=pd.read_csv('./DataSet/Train.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (746,835) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# we will copy this train data for reference
train_orig=train.copy()

<h5> EDA 1 : Lets drop duplicates

In [4]:
# we have 2395 columns .  Lets drop duplicates and see number of columns after that
train=train.T.drop_duplicates()
train=train.T

In [5]:
train.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,RIGHGSN,1,10867.1,8648.34,15766.2,28904.7,5273.65,23461.5,12856,28715.1,...,0,1,0,1,214,214,68,22,22,7
1,RIMHI5ZGP,0,258431,217963,186629,186629,215452,192362,172654,166697,...,0,0,0,0,41,41,38,12,12,11
2,RIHOG1SZU,0,1.45935e+07,1.26006e+07,1.10031e+07,1.01031e+07,1.87988e+07,1.51404e+07,5.32078e+06,6.90632e+06,...,0,1,1,0,378,378,310,21,21,16
3,RIHOLMQWU,0,672704,818112,679489,679489,825725,760580,357713,423661,...,0,0,0,0,702,702,644,951,951,743
4,RIHO584ET,0,-1.21874e+06,-3.00536e+06,-166624,-2.40357e+06,2.58318e+06,2.08655e+06,5.93836e+06,5.44653e+06,...,0,0,0,0,1305,1305,1211,2205,2205,1831


In [6]:
# we will separate out Col1,Col2 before doing further EDA
train.drop(columns=['Col1','Col2'],axis=1,inplace=True);

In [7]:
# Corr wants all column voalues to be numeric. But doing that we got some error.
# So , lets see which columns gave problem converting 

for column in train:
    try:
        train[column].astype(float);
    except Exception:
        print(column)

Col747
Col836


In [8]:
train[['Col747','Col836']]

,Col747,Col836
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


<h5> EDA 2 : Drop columns whose NaN is more than 30 percent

In [9]:
# get columns where NaN are more than 30 percent
for column in train:
    if (round((train[column].isnull().sum() * 100/ len(train[column])),2))>50:
        train.drop(column, axis=1, inplace=True)
    

In [10]:
train.head()

,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,...,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,10867.1,8648.34,15766.2,28904.7,5273.65,23461.5,12856,28715.1,9114.59,30430.8,...,0,1,0,1,214,214,68,22,22,7
1,258431,217963,186629,186629,215452,192362,172654,166697,201352,185940,...,0,0,0,0,41,41,38,12,12,11
2,1.45935e+07,1.26006e+07,1.10031e+07,1.01031e+07,1.87988e+07,1.51404e+07,5.32078e+06,6.90632e+06,9.36834e+06,1.11333e+07,...,0,1,1,0,378,378,310,21,21,16
3,672704,818112,679489,679489,825725,760580,357713,423661,430485,406554,...,0,0,0,0,702,702,644,951,951,743
4,-1.21874e+06,-3.00536e+06,-166624,-2.40357e+06,2.58318e+06,2.08655e+06,5.93836e+06,5.44653e+06,3.47344e+06,3.26217e+06,...,0,0,0,0,1305,1305,1211,2205,2205,1831


<h5> EDA 3 : Check Constant columns

In [11]:
# check any contant columns are there
train.columns[train.nunique()==1 ]


Index([], dtype='object')

So, we dont have any constant columns. Nothing to do with it for this data set

<h5> EDA 4 : Corelation matrix and rop highl corelated columns

In [12]:
# Create correlation matrix
corr_matrix = train.astype(float).corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))



list the columns to dropped who has more than 90 percent corelation with another column.   
So, if colmn 1 and coln 2 has corelation more than 90 percent, we will keep colmn 1 and we will drop column2

In [13]:
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.70)]

In [14]:
len(to_drop)

1583

In [15]:
train.drop(to_drop,axis=1,inplace=True)

In [16]:
train.head()

,Col3,Col9,Col13,Col14,Col17,Col18,Col21,Col22,Col23,Col24,...,Col1383,Col1386,Col1387,Col1388,Col1389,Col1527,Col1575,Col1589,Col1617,Col1631
0,10867.1,12856,9.2935,8.57048,0.548538,0.403366,1.48652,1.72832,0.993441,1.29705,...,0,0,0,1,1,0,0,0,0,0
1,258431,172654,12.4624,12.2805,1.1679,2.6764,0.792123,0.934556,0.893203,0.96662,...,0,0,0,0,0,0,0,0,0,0
2,1.45935e+07,5.32078e+06,16.4961,16.7493,1.14518,2.20387,0.422264,0.498347,0.683582,0.735339,...,0,0,0,0,1,0,0,0,0,0
3,672704,357713,13.4191,13.624,1.20401,0.536304,0.437243,0.521341,0.623499,0.534531,...,0,0,0,0,0,1,0,0,1,0
4,-1.21874e+06,5.93836e+06,NaN,14.7645,NaN,NaN,NaN,1.34464,NaN,1.56343,...,0,2,1,0,0,1,0,1,1,0


<h5> So , now we have only 414 columns .  <p>
we will do our first model based on it .  <p> 
Later we will do more EDA , like can we group more columns to a new columns/feature ?

In [17]:
X=train

In [18]:
train_filtered_columns=train.columns

In [19]:
len(train_filtered_columns)

404

In [20]:
# we will put Col2 , thats target back to train
y=train_orig['Col2']

In [21]:
X.shape

(17521, 404)

In [22]:
y.shape

(17521,)

Lets shuffle the data , create two sets train and test and check the score.  
No need to shuffle explicitly, sklearn train_test_spli does shuffle by default

In [62]:
#y = train.target
#X = train.drop(['target'], axis=1)

In [63]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [64]:
#model = XGBClassifier()

In [65]:
#model.fit(X_train.astype(float),y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [66]:
#y_pred = model.predict(X_test.astype(float))
#predictions = [round(value) for value in y_pred]

In [67]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.27%


In [68]:
# what is the accuracy score on train data itself ?
y_pred_on_train = model.predict(X_train.astype(float))
predictions_onTrain = [round(value) for value in y_pred_on_train]


In [69]:
accuracy_onTrain = accuracy_score(y_train, predictions_onTrain)
print("accuracy_onTrain: %.2f%%" % (accuracy_onTrain * 100.0))

accuracy_onTrain: 90.91%


lets do to submit our model on test data set

In [23]:
test_data=train=pd.read_csv('./DataSet/Test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (700,731,740,752,761,789,811,820,829,841,850) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
test_data.head(5)

,Col1,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,...,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,RIGD58ZWD,8.167140e+04,8.614542e+04,7.532296e+04,7.532296e+04,1.125174e+05,1.002778e+05,2.038915e+04,3.053853e+04,8.395911e+04,...,0,0,0,0,44.0,44.0,23.0,41.0,41.0,36.0
1,RIH660YDS,-1.362824e+07,-4.493043e+06,-3.777257e+06,-4.493043e+06,-4.749764e+06,-4.749764e+06,1.147556e+07,1.147556e+07,1.272055e+07,...,1,1,2,2,1234.0,1234.0,680.0,154.0,153.0,86.0
2,RIH660Q96,1.302700e+06,1.341449e+06,1.240085e+06,1.341449e+06,1.477503e+06,1.477503e+06,3.904086e+05,3.904086e+05,4.979106e+05,...,0,0,0,0,33.0,24.0,19.0,3.0,1.0,1.0
3,RIYDO15W1,1.932258e+05,1.454433e+05,1.929148e+05,1.454433e+05,1.318250e+04,1.318250e+04,8.650108e+04,8.650108e+04,5.896693e+03,...,0,0,0,0,8.0,8.0,3.0,7.0,7.0,3.0
4,RIYBGC1ZD,3.913385e+05,1.987337e+05,1.703432e+05,1.987337e+05,3.046536e+05,3.046536e+05,2.528307e+05,2.528307e+05,3.034449e+05,...,0,0,0,0,7.0,7.0,7.0,6.0,6.0,5.0


In [25]:
# keep the col1 separately
col1_test=test_data['Col1']

In [26]:
train_filtered_columns_list=list(train_filtered_columns)

In [27]:
# now get only those column that are in train data.
test_model= test_data[train_filtered_columns_list]

In [28]:
test_model.shape

(20442, 404)

In [29]:
X.shape

(17521, 404)

In [30]:
model = XGBClassifier()

In [31]:
model.fit(X.astype(float),y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [32]:
y_pred = model.predict(test_model.astype(float))

In [188]:
#join col1_test and y_pred

In [33]:
test_data['traget']=y_pred

In [34]:
test_data.traget.unique()

array([0, 1])

In [35]:
test_results_to_submit=pd.DataFrame(columns={'Col1', 'Col2'})
test_results_to_submit[['Col1', 'Col2']] = test_data[['Col1', 'traget']]

In [36]:
test_results_to_submit.to_csv("prediction_to_submit3.csv")